In [1]:
import pandas as pd

In [2]:
df = pd.ExcelFile('backtest_transactions.xlsx')

In [3]:
df.sheet_names

['Sheet1']

In [4]:
df = df.parse(sheet_name='Sheet1')

In [5]:
df.columns

Index(['Date', 'TradingSymbol', 'Qty', 'TransactionType', 'EntryTime',
       'EntryPrice', 'ExitTime', 'ExitPrice', 'StopLoss', 'Target',
       'TargetSLType', 'TradeCycle', 'OrderExitType', 'OrderStatus', 'PNL',
       'EntryDate', 'SquareOffDate'],
      dtype='object')

In [6]:
df['timestamp'] = pd.to_datetime(df['EntryTime'], format="%Y-%m-%dT%H:%M:%S%z")

In [7]:
df['timestamp']

0      2023-03-02 09:21:00+05:30
1      2023-03-02 09:25:00+05:30
2      2023-03-03 09:21:00+05:30
3      2023-03-03 09:31:00+05:30
4      2023-03-06 09:20:00+05:30
                  ...           
1768   2025-02-24 13:38:00+05:30
1769   2025-02-25 09:32:00+05:30
1770   2025-02-25 09:46:00+05:30
1771   2025-02-27 09:24:00+05:30
1772   2025-02-27 09:38:00+05:30
Name: timestamp, Length: 1773, dtype: datetime64[ns, UTC+05:30]

In [9]:
df['year'] = df['timestamp'].dt.year
df['month'] = df['timestamp'].dt.month
df['day'] = df['timestamp'].dt.day
df['hour'] = df['timestamp'].dt.hour
df['minute'] = df['timestamp'].dt.minute
df['second'] = df['timestamp'].dt.second
df['day_of_week'] = df['timestamp'].dt.day_name()

In [10]:
df = df[df["OrderStatus"]=="COMPLETED"]
manav = df[['year', 'month', 'day', 'hour', 'minute', 'day_of_week', 'PNL']]

In [11]:
manav.to_csv("manav.csv")

In [10]:
analyze = df.groupby(['year', 'month'])[['PNL']].sum()

In [11]:
analyze.to_csv("analysis_0.csv")

In [12]:
analyze = df.groupby(['year', 'month', 'hour'])[['PNL']].sum()

In [13]:
analyze.to_csv("analysis.csv")

In [ ]:
analyze = df.groupby(['year', 'month', 'hour'])[['PNL']].sum()